In [13]:
import pandas as pd

input_path = '../Data/Intermediate_Files/'
clinical_data_path = '../Data/Raw_Data/Clinical_Data/'
output_path = '../Data/Processed_Data/'


df_methyl = pd.read_pickle(
    input_path+'df_batch_uncorrected.pkl').sort_index()

print(
    f' Dataset (df) contains {df_methyl.shape[1]} columns (5mC nucleotides/probes) and {df_methyl.shape[0]} rows (samples).')

# Import functions to clean up clinical data
from FM_Functions.Clinical_Data_CleanUp import *

# Call functions to merge, index and clean clinical data files
labels_0531         = clean_cog       (merge_index_0531())
labels_1031         = clean_cog       (merge_index_1031())
labels_aml05        = clean_aml05     (merge_index_aml05())
labels_beataml      = clean_beataml   (merge_index_beataml())
labels_amltcga      = clean_amltcga   (merge_index_amltcga())
labels_nordic_all   = clean_nordic_all(merge_index_nordic_all())
labels_mds_taml     = clean_mds_taml  (merge_index_mds_taml())
labels_all_graal    = clean_all_graal (merge_index_all_graal())
labels_target_all   = clean_target_all(merge_index_target_all())

# Combine all clinical data labels into one dataframe
labels_combined = pd.concat([labels_aml05, labels_beataml,
                         labels_0531, labels_amltcga, labels_1031,
                        labels_nordic_all, labels_mds_taml,
                        labels_all_graal,labels_target_all], axis=0, join='outer')

# read x_top from output file
x = pd.read_pickle(output_path+'ewas_dmr/ewas_top_x.pkl')

# Remove samples that are not in the methyl dataset
df_labels = labels_combined.loc[labels_combined.index.isin(x.index)].sort_index()

 Dataset (df) contains 333352 columns (5mC nucleotides/probes) and 3330 rows (samples).


In [19]:
# Import functions to clean up clinical data
from FM_Functions.Cox_Lasso_Function import *

# Train Cox-PH model with LASSO
coefs = train_coxph_lasso(df=df_labels, event='os.evnt', time='os.time',
                          train_x=x.iloc[:,:100],cv_n_split=10, loops=10)

Running Cox-Lasso through 10 loops:


  0%|          | 0/10 [00:00<?, ?it/s]/home/fmarchi/projects/MethylScore/Code/.venv_py38/lib/python3.8/site-packages/sklearn/pipeline.py:405: UserWarning: all coefficients are zero, consider decreasing alpha.
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/home/fmarchi/projects/MethylScore/Code/.venv_py38/lib/python3.8/site-packages/sklearn/pipeline.py:405: UserWarning: all coefficients are zero, consider decreasing alpha.
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
 10%|█         | 1/10 [00:00<00:05,  1.74it/s]/home/fmarchi/projects/MethylScore/Code/.venv_py38/lib/python3.8/site-packages/sklearn/pipeline.py:405: UserWarning: all coefficients are zero, consider decreasing alpha.
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/home/fmarchi/projects/MethylScore/Code/.venv_py38/lib/python3.8/site-packages/sklearn/pipeline.py:405: UserWarning: all coefficients are zero, consider decreasing alpha.
  self._final_estimator.fit(Xt, y, **fit_params_last_st

Cox-Lasso Trained Successfuly!
Selected alpha value: [0.021377780404390822]


In [26]:
coefs

,coef_0,coef_1,coef_2,coef_3,coef_4,coef_5,coef_6,coef_7,coef_8,coef_9,nonzero_count,nonzero_freq
IlmnID,,,,,,,,,,,,
cg01543603,0.101331,0.088851,0.086266,0.091520,0.109460,0.089169,0.087532,0.086266,0.085848,0.087532,10,1.111111
cg25292309,-0.058593,-0.051546,-0.051902,-0.050783,-0.060176,-0.054036,-0.051847,-0.051902,-0.052392,-0.051847,10,1.111111
cg20310759,-0.081225,-0.044011,-0.048395,-0.039300,-0.088123,-0.065055,-0.046083,-0.048395,-0.056152,-0.046083,10,1.111111
cg10089193,0.100569,0.074000,0.079050,0.067875,0.105827,0.090012,0.076657,0.079050,0.085329,0.076657,10,1.111111
cg08900363,-0.145072,-0.130750,-0.136417,-0.123897,-0.145841,-0.143864,-0.133704,-0.136417,-0.141368,-0.133704,10,1.111111
...,...,...,...,...,...,...,...,...,...,...,...,...
cg20429981,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
cg12649208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
cg14037250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000


In [25]:
score_train, i = generate_coxph_score(coef_mean=coefs, df=df_labels, x=x.iloc[:,:100],
                                   score_name='MethylScore',train_test='train',
                                   cutoff_train=0.6)

KeyError: 0